<a href="https://colab.research.google.com/github/JoaoVitorSantiagoNogueira/ML2023/blob/main/PokemonClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar as bibliotecas

In [1]:
import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

#Preparar o Dataset


Ler o data set

In [27]:
pokemonDF = pd.read_csv("/content/pokemon.csv")

Préprocessar

## Remover as entradas que não deveriam pertencer ao dataset.

Aqui removemos as linhas que o enunciado solicita que nós não utlizemos.





O que é uma pena, o conhecimento a priório de pokémon diz que além de fraqueza contra elétrico, pokémons aquaticos tem fraqueza contra grama. Essa seria um ótimo parametro para selecionar pokémons como Swampert, Wishcash e Seismictoad, cujos tipos secundários (terra) conferem imunidade ao tipo elétrico.

Além disso, discartar o tipo secundário, resulta em um dataset imperfeito, já que a ordem do tipo do pokémon é inter operavel, vide: https://www.reddit.com/r/pokemon/comments/b2zvom/primary_vs_secondary_types/

 Dois pokémons pode ter o mesmo tipo em ordem reversa, como sableye (Dark/Ghost) e Spiritomb (Ghost/Dark). Além disso, pokémons podem ter o tipo pertinente como o segundo tipo, que são desconsiderados.

Além disso temos um exemplo que pertence a ambas as classes, Bibarel, que tem ambos os tipos, que por nossa classificacao, pertenceria ao tipo normal, mas que igualmente poderia ser classificado como tipo agua. Por ele ser um outlier nesse aspecto, ele foi removido.

In [29]:
#remover bibarel
pokemonDF = pokemonDF.drop(pokemonDF[pokemonDF['pokedex_number'] == 400].index,)

In [30]:
# remover linhas "ilegais"
pokemonDF = pokemonDF.drop(["abilities", "against_bug", "against_dark",	"against_dragon",	"against_fairy",	"against_fight",	"against_fire",	"against_flying",	"against_ghost", "against_grass","against_ground","against_ice","against_normal","against_poison","against_psychic","against_rock", "against_steel", "against_water", "type2"], axis =1,errors='ignore')


Além disso, alguns dados são irrelevantes ou difíceis de serem utilizados. Alguns exemplos são nomes, classificacao e numero no pokedex. Nome e classificacao podem dar pistas por trás do tipo de pokemon, por exemplo o pokémon seaking contém a palavra "sea" no nome, e é classificado como o goldfish pokémon. Quanto ao número e geracao, são apenas ordenacoes arbitrárias que näo tem muita relacao com o pokemon em si.

In [31]:
pokemonDF = pokemonDF.drop(["classfication", "name", "japanese_name", "pokedex_number", "generation"], axis = 1, errors='ignore')

Outras váriaveis como se um pokémon é lendario ou não, soma total de status, e altura provavlemente não são relevantes, mas na incerteza manteremos esses dados e faremos uma analise futura.
Algumas caracteristicas que nosso conhecimento prévio nos diz que provavelmente são bons indicadores para o tipo são a fraqueza a eleétrico, já que quase todo pokemon tipo agua tem, além de fatores como os stats do pokémon (vida, defesa etc) que suspeitamos que os pokémons tipo agua tenham mais vida e ataque especial, e os tipo normal ataque e velocidade.

In [32]:
pokemonDF

,against_electric,attack,base_egg_steps,base_happiness,base_total,capture_rate,defense,experience_growth,height_m,hp,percentage_male,sp_attack,sp_defense,speed,type1,weight_kg,is_legendary
0,0.5,49,5120,70,318,45,49,1059860,0.7,45,88.1,65,65,45,grass,6.9,0
1,0.5,62,5120,70,405,45,63,1059860,1.0,60,88.1,80,80,60,grass,13.0,0
2,0.5,100,5120,70,625,45,123,1059860,2.0,80,88.1,122,120,80,grass,100.0,0
3,1.0,52,5120,70,309,45,43,1059860,0.6,39,88.1,60,50,65,fire,8.5,0
4,1.0,64,5120,70,405,45,58,1059860,1.1,58,88.1,80,65,80,fire,19.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
796,2.0,101,30720,0,570,25,103,1250000,9.2,97,NaN,107,101,61,steel,999.9,1
797,0.5,181,30720,0,570,255,131,1250000,0.3,59,NaN,59,31,109,grass,0.1,1
798,0.5,101,30720,0,570,15,53,1250000,5.5,223,NaN,97,53,43,dark,888.0,1
799,1.0,107,30720,0,600,3,101,1250000,2.4,97,NaN,127,89,79,psychic,230.0,1


vamos separar em dois dataset também. O exercicio pede um classifcador binário, normal ou água, mas poderiamos ter um terceiro fator, nenhum para os demais pokémons. Por isso vamos criar um o dataframe normal e agua, DFNW.

In [38]:
#remover não tipos agua ou normal.
DFWN = pokemonDF.drop(pokemonDF[((pokemonDF['type1'] != "water") & (pokemonDF['type1'] != "normal"))].index,)
DFWN

,against_electric,attack,base_egg_steps,base_happiness,base_total,capture_rate,defense,experience_growth,height_m,hp,percentage_male,sp_attack,sp_defense,speed,type1,weight_kg,is_legendary
6,2.0,48,5120,70,314,45,65,1059860,0.5,44,88.1,50,64,43,water,9.0,0
7,2.0,63,5120,70,405,45,80,1059860,1.0,59,88.1,65,80,58,water,22.5,0
8,2.0,103,5120,70,630,45,120,1059860,1.6,79,88.1,135,115,78,water,85.5,0
15,2.0,45,3840,70,251,255,40,1059860,0.3,40,50.0,35,35,56,normal,1.8,0
16,2.0,60,3840,70,349,120,55,1059860,1.1,63,50.0,50,50,71,normal,30.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,1.0,95,30720,0,570,3,95,1250000,2.3,95,NaN,95,95,95,normal,100.5,0
774,1.0,115,5120,70,480,45,65,1250000,0.4,65,50.0,75,95,65,normal,19.9,0
778,2.0,105,3840,70,475,80,70,1000000,0.9,68,50.0,70,70,92,water,19.0,0
779,0.5,60,5120,70,485,70,85,1000000,3.0,78,50.0,135,91,36,normal,185.0,0


## Analise dos dados

#Aprendizado

Vizualizacão